#### Find smaller model for classification.
#### Think about generating dataset for fine-tunning

In [1]:
from langfuse.callback import CallbackHandler
langfuse_handler = CallbackHandler(
    public_key="key",
    secret_key="key",
    host="https://cloud.langfuse.com"
)

In [2]:
from langchain_community.chat_models import ChatLlamaCpp 
from langchain_community.llms import LlamaCpp
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from transformers import AutoModelForCausalLM, AutoTokenizer

callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

# Path to the model weights
models_directory = '/home/arseniy/Документы/Projects/Python/LLM_project/models/'

local_model = models_directory + 'sqlcoder-7b-2.Q4_K_M.gguf' 

llm = ChatLlamaCpp(
    temperature = 0.1,
    #cache = True,
    model_path = local_model,
    n_ctx=3500,
    #top_k = 10,
    n_gpu_layers=33,
    use_mlock = True,
    #tokenizer = AutoTokenizer.from_pretrained("PipableAI/pip-sql-1.3b"),
    versbose = True,
    callback_manager=callback_manager, 

)


/home/arseniy/.var/app/org.jupyter.JupyterLab/config/jupyterlab-desktop/jlab_server/envs/llamaCuda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
llama_model_loader: loaded meta data with 28 key-value pairs and 291 tensors from /home/arseniy/Документы/Projects/Python/LLM_project/models/sqlcoder-7b-2.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = models
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32       

In [3]:
with open('/home/arseniy/Документы/Projects/Python/LLM_project/db_files/raw-small.txt', 'r') as file:
    schema = file.read()

In [4]:
pip_template = """<schema>{schema}</schema>
<question>{question}</question>
<sql>"""

sqlcoder_template = '''
Generate a SQL query to answer [QUESTION]{question}[/QUESTION]

### Database Schema
The query will run on a database with the following schema:
{schema}

### Answer
Given the database schema, here is the SQL query that [QUESTION]{question}[/QUESTION]
[SQL]

'''
prompt_template = PromptTemplate.from_template(sqlcoder_template)


In [5]:
chain = LLMChain(prompt=prompt_template, llm=llm)

/home/arseniy/.var/app/org.jupyter.JupyterLab/config/jupyterlab-desktop/jlab_server/envs/llamaCuda/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [6]:
import sqlvalidator
import sqlcorrect as sc

def main():
    print("🔴 Simple database chatbot app. Type 'exit' to quit.")
    while True:
        # Get user input
        question = input("🟢 Enter your question: ")
        if question.lower() == 'exit':
            print("Exiting the app.")
            break 

        #response = chain.run(schema=description, prompt = question)
        response = chain.invoke({"schema": schema, "question": question},config={"callbacks": [langfuse_handler]})
        

if __name__ == "__main__":
    main()

🔴 Simple database chatbot app. Type 'exit' to quit.


🟢 Enter your question:  what is the amount of projects done by emoloyee with id 5


Langfuse was not able to parse the LLM model. The LLM call will be recorded without model name. Please create an issue so we can fix your integration: https://github.com/langfuse/langfuse/issues/new/choose


 SELECT COUNT(p.id) AS project_count FROM employees e JOIN "mysProjects" p ON e.id = p.employee_id WHERE e.id = 5;


llama_print_timings:        load time =     550.76 ms
llama_print_timings:      sample time =      27.41 ms /    39 runs   (    0.70 ms per token,  1423.10 tokens per second)
llama_print_timings: prompt eval time =   27222.11 ms /   407 tokens (   66.88 ms per token,    14.95 tokens per second)
llama_print_timings:        eval time =    6196.81 ms /    38 runs   (  163.07 ms per token,     6.13 tokens per second)
llama_print_timings:       total time =   33551.11 ms /   445 tokens


🟢 Enter your question:  exit


Exiting the app.
